In [1]:
import csv
import pandas as pd

In [2]:
HOURSINAYEAR = 8766

In [3]:
def dprod(d1, d2, start_yr=""):
    if start_yr!="":
        return {yr: val * d2[yr] for yr, val in d1.items() if yr in d2 and yr >= start_yr}
    else:
        return {yr: val * d2[yr] for yr, val in d1.items() if yr in d2}

"""
dsum() gives a dictionary = key-wise (by-year) sum of given dicts
Note that for this analysis the given dicts will usually have equal key sets
"""
def dsum(d1, d2):
    return {yr: val + d2[yr] for yr, val in d1.items() if yr in d2}


In [4]:
def carbon_calc(lcoe_data):
    print('disc ',lcoe_data.esc['disc'])
    delta = lcoe_data.esc["disc"]   # discount schedule
    Pi = lcoe_data.esc["inf"]       # inflation factors
    t_1_yr = lcoe_data.inp["period_0_yr"] + 1
    # sum of year-wise product of discount & inflation factors, beginning in online year    
    disc_inf = sum(dprod(delta, Pi, start_yr=t_1_yr).values())  

    scc = lcoe_data.inp["scc schedule"]
    # sum of year-wise product of discount & inflation factors with SCC schedule, beginning in online year    
    disc_inf_scc = sum(dprod(delta, dprod(Pi, scc, start_yr=t_1_yr), \
                             start_yr=t_1_yr).values())

    # effective scc
    lcoe_data.other["effective scc"] = disc_inf_scc / disc_inf
    
    # if we are dealing with a renewable-thermal hybrid...
    if lcoe_data.backup is not None:
        print('backup_disc ',lcoe_data.backup.esc['disc'])
        # need to divide by disc_inf to get the raw output in MWh
        raw_output_main = lcoe_data.other["output"] / disc_inf
        raw_output_backup = lcoe_data.backup.other["output"] / disc_inf
        em_main = lcoe_data.inp["kgCO2 per MWh"]
        em_backup = lcoe_data.backup.inp["kgCO2 per MWh"]
        scalar = ((em_main * raw_output_main) + (em_backup * raw_output_backup)) / \
                 (raw_output_main + raw_output_backup)
        return scalar * disc_inf_scc / (disc_inf * 10**4) 
    else:
        return lcoe_data.inp["kgCO2 per MWh"] * disc_inf_scc / (disc_inf * 10**4) 
    # em/10^3 is tonsCO2 per MWh, so em*scc/10^3 is $ per MWh,
    # so em*scc/10^6 is $ per kWh, so em*scc/10^4 is cents per kWh

"""
carbon_calc() determines the methane cost per kWh generated for gas generation
Inputs:
    'lcoe_data' is the object with all LCOE-related data, including inputs,
        escalation factors, and cash flows
Output:
     methane-cost in cents per kWh generated
"""
def methane_calc(lcoe_data):
    delta = lcoe_data.esc["disc"]   # discount schedule
    Pi = lcoe_data.esc["inf"]       # inflation factors
    t_1_yr = lcoe_data.inp["period_0_yr"] + 1
    # sum of year-wise product of discount & inflation factors, beginning in online year    
    disc_inf = sum(dprod(delta, Pi, start_yr=t_1_yr).values())  

    scm = lcoe_data.inp["scm schedule"]
    # sum of year-wise product of discount & inflation factors with SCM schedule, beginning in online year    
    disc_inf_scm = sum(dprod(delta, dprod(Pi, scm, start_yr=t_1_yr), \
                             start_yr=t_1_yr).values())

    # effective scm
    lcoe_data.other["effective scm"] = disc_inf_scm / disc_inf
    
    # Need to use 10^(-9) not 10^(-7) because of how I code percentages
    if lcoe_data.backup is not None: # hybrid generation
        #if (lcoe_data.backup.inp["gen_type"][:3]).lower()=="gas":
            leak_backup = lcoe_data.backup.inp["leak_rate"]
            leak = leak_backup * __backup_multiplier(lcoe_data) 
            heat_rate = lcoe_data.backup.inp["heat_rate"]
            ##mcon = lcoe_data.backup.inp["methane content"]
            ##return (leak*scm*(10**(-11))*0.969*mcon*19.3*heat_rate)
            scalar = (leak*(10**(-9))*0.969*19.3*heat_rate)
            return scalar * disc_inf_scm / disc_inf  
        #else:
            #return 0.0
    else:   # single-source generation
        if (lcoe_data.inp["gen_type"][:3]).lower()=="gas":
            leak = lcoe_data.inp["leak_rate"]
            heat_rate = lcoe_data.inp["heat_rate"]
            ##mcon = lcoe_data.inp["methane content"]
            ##return (leak*scm*(10**(-11))*0.96899*mcon*19.3*heat_rate)
            scalar = (leak*(10**(-9))*0.96899*19.3*heat_rate)
            return scalar * disc_inf_scm / disc_inf  
        else:
            return 0.0

    
"""
non_carbon_calc() determines the non-carbon cost per kWh, mostly to be used
    for hybrid generation
Inputs:
    'lcoe_data' is the object with all LCOE-related data, including inputs,
        escalation factors, and cash flows
Output:
     non-carbon pollution/health costs in cents per kWh generated        
"""
def non_carbon_calc(lcoe_data):
    ncc_main = lcoe_data.inp["non_carbon_c"]["mean"]
    if lcoe_data.backup is not None:
        ncc_backup = lcoe_data.backup.inp["non_carbon_c"]["mean"]
        return ncc_backup * __backup_multiplier(lcoe_data) + \
               ncc_main * __main_multiplier(lcoe_data)  
    else:
        return ncc_main  


"""
transmission_calc() determines the transmission cost per kWh, specifically
for hybrid generation
"""
def transmission_calc(lcoe_data):
    if lcoe_data.backup is not None:
        # for the backup
        gb = lcoe_data.backup.inp["trans_cost"]
        pb = lcoe_data.backup.inp["orig_cap_fac"] 
        Cb = lcoe_data.backup.inp["cap"]
        pprime = lcoe_data.backup.inp["cap_fac"] 
        # for the renewable source
        gr = lcoe_data.inp["trans_cost"]
        pr = lcoe_data.inp["cap_fac"] 
        Cr = lcoe_data.inp["cap"]

        T = lcoe_data.inp["tax"] 

        return (gb * pb * Cb + gr * pr * Cr) / (pprime * Cb + pr * Cr)  
    else:
        return lcoe_data.inp["trans_cost"]  

# Helper functions
def __backup_multiplier(lcoe_data):
    actual_cap_backup = lcoe_data.backup.inp["cap"] * \
                        lcoe_data.backup.inp["cap_fac"] 
    actual_cap_renew = lcoe_data.inp["cap"] * lcoe_data.inp["cap_fac"] 
    return actual_cap_backup / (actual_cap_backup + actual_cap_renew)

def __main_multiplier(lcoe_data):
    # NOTE: I think this multiplier should be the following:
    actual_cap_backup = lcoe_data.backup.inp["cap"] * \
                        lcoe_data.backup.inp["cap_fac"] 
    actual_cap_renew = lcoe_data.inp["cap"] * lcoe_data.inp["cap_fac"] 
    return actual_cap_renew / (actual_cap_backup + actual_cap_renew)




In [5]:
def lcoe_calc(lcoe_data):
    # We first calculate the Total NPV of Costs. Then we calculate Output
    # and determine LCOE from the scaled start-year Price.
    costs = __total_npv_costs(lcoe_data)        # this is a dict
    output = __total_output(lcoe_data)          # this is a float
    # Divide by 10 to go from $/MWh to cents/kWh
    # The order of the following two commands MUST be maintained!
    lcoe_data.lcoe = { k: v/output/10 for k, v in costs.items() }
    b = sum(lcoe_data.lcoe.values())
    lcoe_data.lcoe["base no transmission"] = b
    lcoe_data.lcoe["Source name"] = lcoe_data.inp["gen_type"]
    lcoe_data.lcoe["Transmission"] = transmission_calc(lcoe_data)
    lcoe_data.lcoe["LCOE Base"] = round(b + lcoe_data.lcoe["Transmission"], 4)
    lcoe_data.lcoe["Carbon"] = carbon_calc(lcoe_data)
    lcoe_data.lcoe["Methane"] = methane_calc(lcoe_data)
    lcoe_data.lcoe["GHG External Costs"] = lcoe_data.lcoe["Carbon"] + lcoe_data.lcoe["Methane"]
    lcoe_data.lcoe["Non-GHG External Costs"] = non_carbon_calc(lcoe_data)
    lcoe_data.lcoe["Online year"] = lcoe_data.inp["period_0_yr"] + 1
    lcoe_data.lcoe["Effective SCC"] = round(lcoe_data.other["effective scc"], 2)
    lcoe_data.lcoe["Effective SCM"] = round(lcoe_data.other["effective scm"], 2)
    lcoe_data.lcoe["summary"] = \
        { "LCOE Base no transmission": round(b, 4), \
          "LCOE Base with transmission": round(b + \
                                               lcoe_data.lcoe["Transmission"], \
                                               4), \
          "LCOE w/ Carbon": round(b + \
                                  lcoe_data.lcoe["Carbon"] + \
                                  lcoe_data.lcoe["Transmission"], 4), \
          "LCOE w/ Carbon & Methane": round(b + \
              lcoe_data.lcoe["Carbon"] + lcoe_data.lcoe["Methane"] + \
              lcoe_data.lcoe["Transmission"], 4), \
          "LCOE w/ Total Social Costs": round(b + \
              lcoe_data.lcoe["Carbon"] + lcoe_data.lcoe["Methane"] + \
              lcoe_data.lcoe["Transmission"] + \
              lcoe_data.lcoe["Non-GHG External Costs"], 4), \
          }

# ------------------------------------------------------------------------ #
"""
Private helper function that computes the total net present value of costs
Refer to the mathematical appendix.

Returns a dictinoary of costs by cost category
"""
def __total_npv_costs(lcoe_data):
    # Variables used in the model
    C = lcoe_data.inp["cap"]        # capacity
    F = lcoe_data.inp["fl_c"]       # marginal fuel cost in start year
    H = lcoe_data.inp["heat_rate"] 
    O = lcoe_data.inp["on_c"]       # overnight cost
    T = lcoe_data.inp["tax"]        # tax rate
    sigma = lcoe_data.inp["fx_om_c"]    # fixed O&M costs
    mu = lcoe_data.inp["vr_om_c"]       # variable O&M costs
    p = lcoe_data.inp["cap_fac"]    # plant's capacity factor
    w = lcoe_data.inp["waste_fee"]  # nuc only
    
    delta = lcoe_data.esc["disc"]   # discount schedule
    Pi = lcoe_data.esc["inf"]       # inflation factors
    Omega = lcoe_data.esc["om"]     # O&M cost increase factors
    Phi = lcoe_data.esc["fl"]       # Fuel cost increase factors
    
    d = lcoe_data.inp["depr"]       # corrected depreciation schedule
    kappa = lcoe_data.inp["cnstr"]  # corrected construction schedule

    t_1_yr = lcoe_data.inp["period_0_yr"] + 1
    end = lcoe_data.inp["end_yr"]

    # Various scalars & coefficients used in the model
        # dlt = delta   k = kappa   p = Pi      Refer to Section 2 of math doc
    # I prefer using 'sum' w/ 'dprod' for summations, as opposed to 'for' loops
    dlt = sum([v for yr, v in delta.items() if yr >= t_1_yr])
    dlt_d = sum(dprod(delta, d).values())
    dlt_k_p = sum(dprod(dprod(delta, kappa), Pi).values())
    k_p = sum(dprod(kappa, Pi).values())
    om_scalar = sum(dprod(delta, Omega, start_yr=t_1_yr).values())
    fuel_scalar = sum(dprod(delta, Phi, start_yr=t_1_yr).values())
    disc_inf = sum(dprod(delta, Pi, start_yr=t_1_yr).values())   
    is_nuc = 1 if (lcoe_data.inp["gen_type"][:3] == "nuc") else 0 
    
    # Capital = construction, depreciation, and decommissioning costs, in that order
    costs = {}
    costs["Capital"] = C * O * dlt_k_p - \
                       C * O * T * dlt_d * k_p + \
                       delta[end] * (1 - T) * O * 175 * Pi[end] * is_nuc
    costs["Fixed O&M"] = (1 - T) * C * sigma * om_scalar
    costs["Variable O&M"] = (1 - T) * C * p * mu * om_scalar
    # fuel costs and waste fee, in that specific order.
    # the 10**6 is to adjust for the fact that waste & fuel have different units
    costs["Fuel"] = (1 - T) * C * p * F * H * fuel_scalar + \
                    (1 - T) * dlt * C * p * w * 10**6 * is_nuc

    # Account for the units multipliers
    units_mult = {}
    units_mult["Capital"] = 10**(-3) 
    units_mult["Fixed O&M"] = 10**(-3)
    units_mult["Variable O&M"] = HOURSINAYEAR * 10**(-6)
    units_mult["Fuel"] = HOURSINAYEAR * 10**(-9)

    costs = dprod(costs, units_mult)
    
    if lcoe_data.backup is not None:
        # note the recursion
        costs = dsum(costs, __total_npv_costs(lcoe_data.backup))
        print("inp: ",lcoe_data.backup.inp)
        print("esc：",lcoe_data.backup.esc)
    return costs  

    
"""
Private helper function that computes the total post-tax discounted output

Returns a float equal to the total discounted post-tax output
"""
def __total_output(lcoe_data):
    # Variables used for computing output
    C = lcoe_data.inp["cap"]                # capacity
    p = lcoe_data.inp["cap_fac"]            # capacity factor
    T = lcoe_data.inp["tax"]                # tax rate
    delta = lcoe_data.esc["disc"]           # discount schedule
    Pi = lcoe_data.esc["inf"]               # inflation factors
    # Sum of the year-wise product of the discount & inflation factors
    disc_inf = sum(dprod(delta, Pi, \
        start_yr=lcoe_data.inp["period_0_yr"] + 1).values())

    output = (1 - T) * C * p * disc_inf * HOURSINAYEAR * 10**(-6)
    lcoe_data.other["output"] = output

    if lcoe_data.backup is not None:
        output += __total_output(lcoe_data.backup)  # note the recursion
        print("backup op", __total_output(lcoe_data.backup))
        print(output)
    return output


In [6]:
def escalation(lcoe_data, fl_shift_yr=2040, fl_shift_fls=['coal', 'gas', 'nuclear']):
    # for the backup power source's escalation, we carefully use recursion
    if lcoe_data.backup is not None:
        escalation(lcoe_data.backup)
    # core_gen_type is "coal" if the input generation type is "Coal IGCC"
    core_gen_type = lcoe_data.inp["gen_type"].partition(' ')[0].lower()
    esc = {}
    rates = {}
    start = lcoe_data.inp["start_yr"]
    end = lcoe_data.inp["end_yr"]
    p0yr = lcoe_data.inp["period_0_yr"]
    
    # inflation. the /100.0 are for percentage correction
    rates["inf"] = 1 + (lcoe_data.inp["inf"]/100.0)
    # note that O&M and Fuel escalation rates account for inflation!
    rates["om"] = (1 + (lcoe_data.inp["inf"]/100.0)) * \
                       (1 + (lcoe_data.inp["om_real"]/100.0))
    if lcoe_data.inp["fuel price schedule"] is None:
        rates["fl"] = (1 + (lcoe_data.inp["inf"]/100.0)) * \
                           (1 + (lcoe_data.inp["fl_real"]/100.0))

    # iterate over three types of escalation factors
    for f in ["inf", "om", "fl"]:
        esc[f] = {}
        if f=="fl" and (core_gen_type in fl_shift_fls):
            # use EIA projection for fuel escalation until fl_shift_yr=2040,
            # then use inflation factor beyond that
            if lcoe_data.inp["fuel price schedule"] is not None:
                # here we have an actual schedule
                base_price = lcoe_data.inp["fuel price schedule"][start]
                shift_price = lcoe_data.inp["fuel price schedule"][fl_shift_yr]
                lcoe_data.inp["fl_c"] = base_price
                for yr in range(start, fl_shift_yr + 1):
                    if base_price != 0:
                        esc[f][yr] = lcoe_data.inp["fuel price schedule"][yr] * \
                                     ( rates["inf"] ** (yr - start) ) / base_price
                    else: # if initial price is 0, escalation values are infinite
                        esc[f][yr] = 0
                for yr in range(fl_shift_yr + 1, end + 1):
                    if base_price != 0:
                        esc[f][yr] = (shift_price / base_price) * \
                                     ( rates["inf"] ** (yr - start) )
                    else: # if initial price is 0, escalation values are infinite
                        esc[f][yr] = 0
            else:
                # we just have a single fuel inflation rate until 2040
                for yr in range(start, fl_shift_yr + 1):
                    esc[f][yr] = rates[f] ** (yr - start)
                for yr in range(fl_shift_yr + 1, end + 1):
                    esc[f][yr] = ( rates[f] ** (fl_shift_yr - start) ) * \
                                 ( rates["inf"] ** (yr - fl_shift_yr) )
        else:
            # simple exponential annual growth in escalation factor
            for yr in range(start, end + 1):
                esc[f][yr] = rates[f] ** (yr - start)
    
    # discount factor using WACC, with baseline year equal to start year
    wacc = lcoe_data.inp["wacc"] / 100.0    # percentage correction
    esc["disc"] = {}
    for pd in range(0, end - start + 1):    # +1 is to include end year
        esc["disc"][pd + start] = 1/(1 + wacc)**(pd + 0)
    
    lcoe_data.esc = esc


In [7]:
def clean_inputs(lcoe_data):
    # recursion base case - parse input and error check
    __parse_numbers([lcoe_data.inp, \
                     lcoe_data.inp["construction schedule"], \
                     lcoe_data.inp["depreciation schedule"], \
                     lcoe_data.inp["non_carbon_c"], \
                     lcoe_data.inp["fuel price schedule"], \
                     lcoe_data.inp["scc schedule"], \
                     lcoe_data.inp["scm schedule"], \
                     ])
    __error_check(lcoe_data)

    if lcoe_data.backup is not None:
        # Note the use of recursion for the backup
        clean_inputs(lcoe_data.backup)
        # We need to calculate the backup source's mean actual capacity factor
        # <note: this command must go after the recursion base case>
        lcoe_data.backup.inp["orig_cap_fac"] = lcoe_data.backup.inp["cap_fac"]
        lcoe_data.backup.inp["cap_fac"] = \
            __compute_mean_cf_backup(lcoe_data.inp, lcoe_data.backup.inp)
        print('backup_cf_mean:',lcoe_data.backup.inp["cap_fac"])

        # We also need to make sure that the years align
        if (lcoe_data.backup.inp["start_yr"] != lcoe_data.inp["start_yr"]) or \
           (lcoe_data.backup.inp["period_0_yr"] != lcoe_data.inp["period_0_yr"]) or \
           (lcoe_data.backup.inp["end_yr"] != lcoe_data.inp["end_yr"]):
            # require NGCT to be constructed by its period_0_yr but not begin
            # production until renewable's period_0_yr
            diff = lcoe_data.inp["period_0_yr"] - lcoe_data.backup.inp["period_0_yr"]
            cnstr_old = lcoe_data.backup.inp["cnstr"]
            depr_old = lcoe_data.backup.inp["depr"]
            # Shift the construction and depreciation schedules as necessary...
            for yr in range(lcoe_data.inp["start_yr"], lcoe_data.inp["end_yr"] + 1):
                try:
                    lcoe_data.backup.inp["cnstr"][yr] = cnstr_old[yr + diff]
                    lcoe_data.backup.inp["depr"][yr] = depr_old[yr + diff]
                except KeyError:
                    lcoe_data.backup.inp["cnstr"][yr] = 0
                    lcoe_data.backup.inp["depr"][yr] = 0
            # adjust start and end years for backup
            lcoe_data.backup.inp["start_yr"] = lcoe_data.inp["start_yr"]
            lcoe_data.backup.inp["period_0_yr"] = lcoe_data.inp["period_0_yr"]
            lcoe_data.backup.inp["end_yr"] = lcoe_data.inp["end_yr"]

        # Finally, we need to replace the wacc and tax-rate
        if lcoe_data.backup.inp["hybrid_tax"] != 0:
            lcoe_data.inp["tax"] = lcoe_data.backup.inp["hybrid_tax"] / 100.0
        if lcoe_data.backup.inp["hybrid_wacc"] != 0:    
            lcoe_data.inp["wacc"] = lcoe_data.backup.inp["hybrid_wacc"]
            # I do the WACC pct -> decimal adjustment in the 'escalation' file
        
"""
Private function that computes the backup source's mean actual capacity factor
    in the case of hybrid generation
Inputs:
    'main_dict' is the input dictionary for the renewable fuel
    'backup_dict' is the input dictionary for the backup fuel, usually nat gas
"""
def __compute_mean_cf_backup(main_dict, backup_dict):
    actual_equiv_cap = backup_dict["equiv_cap"] * \
                       backup_dict["equiv_cap_fac"] / 100.0 # pct to decimal
    actual_renew_cap = main_dict["cap"] * main_dict["cap_fac"]
    return ( (actual_equiv_cap - actual_renew_cap) / backup_dict["cap"] )


"""
Private function that makes sure certain values are within their proper ranges.
Make sure to call this function AFTER calling __parse_numbers()
Inputs:
    'lcoe_data' is the object with all LCOE-related data, including inputs
"""
def __error_check(lcoe_data):
    if lcoe_data.inp["cap_fac"] <= 0 or lcoe_data.inp["cap_fac"] > 100:
        raise ValueError("Capacity factor out of bounds!")
    else:
        lcoe_data.inp["cap_fac"] /= 100.0   # Correct for percentage
    lcoe_data.inp["tax"] /= 100.0   # Correct for percentage
    

    # fix the format of construction & depreciation schedules so that the keys
    # are years, not periods. Again, the /100.0 is for percentage correction
    cnstr = {}      # corrected construction schedule as fraction/decimal
    depr = {}       # corrected depreciation schedule as fraction/decimal
    p0yr = lcoe_data.inp["period_0_yr"]
    for yr in range(lcoe_data.inp["start_yr"], lcoe_data.inp["end_yr"] + 1):
        try:
            cnstr[yr] = lcoe_data.inp["construction schedule"][yr - p0yr] / 100.0
        except KeyError:
            cnstr[yr] = 0
        try:
            depr[yr] = lcoe_data.inp["depreciation schedule"][yr - p0yr] / 100.0
        except KeyError:
            depr[yr] = 0
    lcoe_data.inp["cnstr"] = cnstr
    lcoe_data.inp["depr"] = depr

    # convert fuel price schedule to have integer year keys
    if lcoe_data.inp["fuel price schedule"] is not None:
        lcoe_data.inp["fuel price schedule"] = __convert_keys_to_int( \
            lcoe_data.inp["fuel price schedule"])

    # clean the carbon and methane schedules
    lcoe_data.inp["scc schedule"] = __convert_keys_to_int( \
        lcoe_data.inp["scc schedule"])
    lcoe_data.inp["scm schedule"] = __convert_keys_to_int( \
        lcoe_data.inp["scm schedule"])


"""
Private function that parses the dictionaries to ensure that numbers have
    the proper type (e.g. 'int' or 'float' instead of 'str')
Inputs:
    'dict_list' is a list of dictionaries to parse
"""
def __parse_numbers(dict_list):
    # All values in these dicts are strings, but we don't want that...
    for d in dict_list:
        if d is not None:
            for k, v in list(d.items()):
                try:
                    if str(k).split('_')[-1] == "yr": # years should be ints
                        d[k] = int(v)
                    else:
                        if v=="":   # missing values treated as 0
                            d[k] = 0.0
                        else:       # typical scenario: cast string -> float
                            d[k] = float(v)
                except (TypeError, ValueError):
                    pass        # do nothing, leave as string


"""
Private function that converts keys of a dictionary to integers
"""
def __convert_keys_to_int(in_dict):
    return { int(k):v for k, v in list(in_dict.items()) }


In [8]:
def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z


In [9]:
def load_inputs_from_csv(lcoe_data, fname, fname_backup="", 
    gnrl_fname="csv_inputs/general", merge=True, index=0):

    try:
        new_general_inputs = list(csv.DictReader(open(gnrl_fname + ".csv")))[index]
        new_inputs_specific = list(csv.DictReader(open(fname + ".csv")))[index]    
        new_cnstr_sched = list(csv.DictReader( \
            open(fname + "_construction_schedule.csv")))[0]
        # NOTE ON CONSTRUCTION SCHEDULE: If a period is not listed in the
        # schedule, it is assumed to be 0 in that period/year. Thus, if the
        # schedule only shows nonzero values for periods -2 to 0, then
        # there is zero construction in period -3. This represents the "lag"
    except IOError:
        pass

    try:
        new_depr_sched = list(csv.DictReader( \
            open(fname + "_depreciation_schedule.csv")))[0]
    except IOError:
        try:
            new_depr_sched = list(csv.DictReader(open(\
                "csv_inputs/DEFAULT_depreciation_schedule.csv")))[0]
        except:
            pass

    try:
        new_fuel_sched = list(csv.DictReader( \
            open(fname + "_fuel_price_schedule.csv")))[0]
    except IOError:
        new_fuel_sched = None

    try:
        new_scc_sched = list(csv.DictReader(open("csv_inputs/CARBON_schedule.csv")))[0]
        new_scm_sched = list(csv.DictReader(open("csv_inputs/METHANE_schedule.csv")))[0]
    except IOError:
        pass
    
    # prioritizes the general variables, which is what we want for now
    new_inputs = merge_two_dicts(new_inputs_specific, new_general_inputs)

    # basic cleaning...
    new_inputs["start_yr"] = int(new_inputs["start_yr"]) 
    new_inputs["period_0_yr"] = int(new_inputs["period_0_yr"])      
    new_inputs["end_yr"] = int(new_inputs["end_yr"])
    for d in [new_cnstr_sched, new_depr_sched]:
        for k in list(d.keys()):
            d[int(k)] = d.pop(k)    # casts all keys as ints, keeps values

    try:    # if file for non-carbon costs exists...
        new_non_carbon_c = list(csv.DictReader( \
            open(fname + "_non_carbon_costs.csv")))[0]
    except: # otherwise use default
        new_non_carbon_c = {"mean":0}

    # if we want backup generation, we need to load-in the backup's data
    if fname_backup!="":    
        # The easiest way to load all of the backup files is recursive:
        # Call load_inputs_from_csv() for the backup object in lcoe_data
        # This is potentially dangerous and can lead to an
        # infinite callback loop if you are not careful
        load_inputs_from_csv(lcoe_data.backup, fname_backup, gnrl_fname=gnrl_fname)
        
        # We need to add the auxiliary data for the backup
        try:
            backup_auxiliaries = list(csv.DictReader( \
                open(fname + "_hybrid_aux.csv")))[0]
        except IOError:
            pass
        lcoe_data.backup.inp.update(backup_auxiliaries)
        
    new_inputs["construction schedule"] = new_cnstr_sched
    new_inputs["depreciation schedule"] = new_depr_sched        
    new_inputs["non_carbon_c"] = new_non_carbon_c
    new_inputs["fuel price schedule"] = new_fuel_sched
    new_inputs["scc schedule"] = new_scc_sched
    new_inputs["scm schedule"] = new_scm_sched

    if merge:
        lcoe_data.inp.update(new_inputs)
    else:
        lcoe_data.inp = new_inputs

In [10]:
class LCOE_Data:
    """
    'gen_type' is the name of the fuel source
    'backup_gen_type' is the name of the backup generation type if we are using
        hybrid generation (e.g. "gas" for solar + gas backup)
    """
    def __init__(self, gen_type, backup_gen_type=""):
        # Dictionary of all inputs for the main power source
        self.inp = {"gen_type": gen_type}
        # Dictionary of escalation factors
        self.esc = {}
        # Dictionary of the results. lcoe["Base"] gives the no-externality #
        self.lcoe = {}

        # Other data
        self.other = {}

        # Backup generation, if needed. Note that it is its own instance
        if backup_gen_type!="":
            # No backup _for_ the backup - this is crucial for recursion
            self.backup = LCOE_Data(backup_gen_type) 
        else:
            self.backup = None

        # set the units as a preliminary
        self.set_units()

    def set_units(self):
        units = {"cap":"MW", "cap_fac":"%", "heat_rate":"Btu/kWh", \
                 "on_c":"$/kW", "inc_cap_c":"$/kW/yr", "fx_om_c":"$/kW/yr", \
                 "vr_om_c":"mill/kWh", "fl_c":"$/mmBtu", "waste_fee":"$/kWh", \
                 "decom_c":"$ million", "growth_rates":"%"}
        self.inp["units"] = units


In [11]:
def lcoe_sequence(gen_type, fpath, backup_gen_type="",
                     backup_fpath="", gnrl_fpath=""):
    dat = LCOE_Data(gen_type, backup_gen_type)      # initiate the LCOE object
    load_inputs_from_csv(dat, fpath, fname_backup=backup_fpath, \
        gnrl_fname=gnrl_fpath)  # loads all inputs
    clean_inputs(dat)   # cleans and error-checks the inputs
    escalation(dat)     # computes the inflation, O&M, fuel, discount factors
    lcoe_calc(dat)      # calls the function below to calculate the LCOEs
    return dat.lcoe

In [12]:
def do_all_lcoes(fnames, backup_fname, gnrl_fpath=""):
    single_src_fnames = [v for k, v in list(fnames.items()) if k < 21]
    renewables = [v for k, v in list(fnames.items()) if k >= 21]
    lcoes = []
    for nm in single_src_fnames:
        # Gets the LCOEs, then appends the dict to the lcoes list
        temp = lcoe_sequence(nm, 'csv_inputs/' + nm, gnrl_fpath=gnrl_fpath)
        # drops 'summary' from temp, now temp2 is 'summary'
        temp2 = temp.pop("summary", None)
        # Adds the full dict of LCOE results to the list
        lcoes.append( { k:v for k, v in (list(temp.items()) + list(temp2.items())) } )
    for nm in renewables:
        temp = lcoe_sequence(nm, 'csv_inputs/' + nm, backup_fname, \
                             'csv_inputs/' + backup_fname, gnrl_fpath=gnrl_fpath)
        temp["Source name"] = nm + ' + NGCT backup'
        temp2 = temp.pop("summary", None)
        lcoes.append( { k:v for k, v in (list(temp.items()) + list(temp2.items())) } )

In [13]:
def do_lcoe():
    fname_dict = {1: 'coal',      2: 'coal with CCS', \
                  3: 'gas',                 4: 'gas with CCS', \
                  5: 'hydro',               6: 'nuclear', \
                  7: 'solar',               8: 'wind', \
                  9: 'offshore wind', \
                  21: 'hydro',              22: 'solar', \
                  23: 'wind',               24: 'offshore wind' \
                  }
    label_dict = {1: 'Conventional coal',   2: 'Coal with 90% CCS', \
                  3: 'Conventional gas',    4: 'Gas with CCS', \
                  5: 'Hydro',               6: 'Nuclear', \
                  7: 'Solar PV',            8: 'Wind', \
                  9: 'Offshore Wind', \
                  21: 'Hydro + NGCT backup', 22: 'Solar PV + NGCT backup', \
                  23: 'Onshore Wind + NGCT backup', 24: 'Offshore Wind + NGCT backup', \
                  }
    do_all_lcoes(fname_dict, backup_fname='gas (advanced ct)', \
                     gnrl_fpath="csv_inputs/general")

    

In [14]:
do_lcoe()

disc  {2019: 1.0, 2020: 0.9389671361502347, 2021: 0.8816592827701736, 2022: 0.8278490918029798, 2023: 0.7773230908948168, 2024: 0.7298808365209549, 2025: 0.6853341187990186, 2026: 0.6435062148347593, 2027: 0.6042311876382717, 2028: 0.5673532278293631, 2029: 0.5327260355205288, 2030: 0.5002122399253792, 2031: 0.46968285439002744, 2032: 0.44101676468547174, 2033: 0.41410024853095945, 2034: 0.38882652444221544, 2035: 0.3650953281147563, 2036: 0.342812514661743, 2037: 0.3218896851283972, 2038: 0.302243835801312, 2039: 0.28379702892141967, 2040: 0.2664760834942908, 2041: 0.2502122849711651, 2042: 0.23494111264898132, 2043: 0.2206019837079637, 2044: 0.2071380128713274, 2045: 0.1944957867336408, 2046: 0.1826251518625735, 2047: 0.17147901583340236, 2048: 0.1610131604069506, 2049: 0.15118606610981278, 2050: 0.14195874752095097, 2051: 0.13329459861122156, 2052: 0.12515924752227378, 2053: 0.11752042020870777}
disc  {2019: 1.0, 2020: 0.9389671361502347, 2021: 0.8816592827701736, 2022: 0.8278490918